# Data Pipeline and Preparation

This notebook covers data loading, data cleaning, feature engineering, and preparation for database storage.


In [1]:
import pandas as pd
import numpy as np



## Load Raw Match Data

The dataset was generated using the API-Football API and contains all completed Premier League matches for the selected seasons.


In [2]:
df = pd.read_csv(
    "data/raw/premier_league_matches_with_stats_3_seasons.csv"
)

df.shape


(1140, 44)

## Initial Data Inspection

We inspect the structure and data types of the dataset.


In [3]:
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Data columns (total 44 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fixture_id             1140 non-null   int64  
 1   date                   1140 non-null   object 
 2   season                 1140 non-null   int64  
 3   home_team              1140 non-null   object 
 4   away_team              1140 non-null   object 
 5   home_goals             1140 non-null   int64  
 6   away_goals             1140 non-null   int64  
 7   venue                  1140 non-null   object 
 8   home_shots_on_goal     1140 non-null   int64  
 9   home_shots_off_goal    1140 non-null   int64  
 10  home_total_shots       1140 non-null   int64  
 11  home_blocked_shots     1140 non-null   int64  
 12  home_shots_insidebox   1140 non-null   int64  
 13  home_shots_outsidebox  1140 non-null   int64  
 14  home_fouls             1140 non-null   int64  
 15  home

## Missing Value Analysis

We identify missing values per variable to decide on appropriate data cleaning strategies.


In [4]:
df.isna().sum().sort_values(ascending=False)


away_red_cards           1013
home_red_cards           1013
away_goals_prevented      750
home_goals_prevented      750
away_expected_goals       187
home_expected_goals       187
home_offsides              50
away_offsides              50
away_yellow_cards          40
home_yellow_cards          40
home_goalkeeper_saves       3
away_goalkeeper_saves       3
home_blocked_shots          0
home_total_shots            0
home_shots_off_goal         0
home_shots_on_goal          0
venue                       0
away_goals                  0
home_goals                  0
away_team                   0
home_team                   0
season                      0
date                        0
fixture_id                  0
home_corner_kicks           0
home_fouls                  0
home_shots_outsidebox       0
home_shots_insidebox        0
away_blocked_shots          0
away_total_shots            0
away_shots_off_goal         0
away_shots_on_goal          0
home_passes_%               0
home_passe

## Removing Variables with Structural Missing Values

Variables with a very high proportion of missing values are removed, as they are not suitable for further analysis.


In [5]:
cols_to_drop = [
    "home_red_cards",
    "away_red_cards",
    "home_goals_prevented",
    "away_goals_prevented"
]

df = df.drop(columns=cols_to_drop)


## Imputation of Expected Goals (xG)

Missing expected goals values are replaced using the median, as xG is a continuous variable.


In [6]:
df["home_expected_goals"] = df["home_expected_goals"].fillna(
    df["home_expected_goals"].median()
)

df["away_expected_goals"] = df["away_expected_goals"].fillna(
    df["away_expected_goals"].median()
)


## Handling Remaining Missing Values

Remaining missing values correspond to event-based statistics and are replaced with zero.


In [7]:
df = df.fillna(0)


In [8]:
excluded_home_cols = {
    "home_team",
    "home_goals",
}


stat_columns = [
    col.replace("home_", "")
    for col in df.columns
    if col.startswith("home_")
    and col not in excluded_home_cols
    and pd.api.types.is_numeric_dtype(df[col])
]


## Final Validation

We ensure that no missing values remain in the dataset.


In [9]:
df.isna().sum().max()

np.int64(0)

## Extraktion der Teams aus dem Datensatz

Im ersten Schritt der Datenbank-Befüllung werden alle eindeutigen Teams aus dem
bereinigten Datensatz extrahiert. Da jedes Match ein Home- und ein Away-Team
enthält, werden beide Spalten kombiniert und dedupliziert.


In [10]:
# Extrahiere alle eindeutigen Teamnamen aus Home- und Away-Teams
home_teams = df["home_team"]
away_teams = df["away_team"]

teams = pd.concat([home_teams, away_teams]).drop_duplicates().sort_values().reset_index(drop=True)

teams_df = pd.DataFrame({"team_name": teams})

display(teams_df.head())
print("Shape:", teams_df.shape)



,team_name
0,Arsenal
1,Aston Villa
2,Bournemouth
3,Brentford
4,Brighton


Shape: (24, 1)


### Erstellung der Tabelle `teams`

Die Tabelle speichert alle Premier-League-Teams eindeutig.
Der Primärschlüssel `team_id` wird automatisch generiert.


In [11]:
create_teams_table = """
CREATE TABLE IF NOT EXISTS teams (
    team_id SERIAL PRIMARY KEY,
    team_name TEXT UNIQUE NOT NULL
);
"""

with open("db/01_create_teams.sql", "w") as f:
    f.write(create_teams_table)

### Einfügen der Teams in die Datenbank

Die extrahierten Teamnamen werden in die Tabelle `teams` eingefuegt.
Durch den UNIQUE-Constraint auf `team_name` werden Duplikate vermieden,
sodass das Notebook mehrfach ausgefuehrt werden kann.


In [12]:
insert_team_sql = """
INSERT INTO teams (team_name)
VALUES (:team_name)
ON CONFLICT (team_name) DO NOTHING;
"""

with open("db/04_insert_teams.sql", "w") as f:
    for _, row in teams_df.iterrows():
        f.write(
            f"INSERT INTO teams (team_name) VALUES ('{row.team_name}') "
            f"ON CONFLICT (team_name) DO NOTHING;\n"
        )



### Erstellung der Tabelle `matches`

Die Tabelle speichert die beteiligten Teams als Namen.
Die Zuordnung zu Team-IDs erfolgt spaeter direkt in der Datenbank.


In [13]:
create_matches_table = """
CREATE TABLE IF NOT EXISTS matches (
    fixture_id INTEGER PRIMARY KEY,
    date DATE NOT NULL,
    season INTEGER NOT NULL,
    venue TEXT,

    home_team TEXT NOT NULL,
    away_team TEXT NOT NULL,

    home_goals INTEGER,
    away_goals INTEGER
);

"""
with open("db/02_create_matches.sql", "w") as f:
    f.write(create_matches_table)

### Vorbereitung der Match-Daten

Aus dem bereinigten DataFrame werden die fuer die Tabelle `matches`
relevanten Spalten ausgewaehlt.


In [14]:
matches_df = df[
    [
        "fixture_id",
        "date",
        "season",
        "venue",
        "home_team",
        "away_team",
        "home_goals",
        "away_goals",
    ]
].drop_duplicates(subset=["fixture_id"])


### Einfügen der Matches

Die Match-Daten werden in die Tabelle `matches` eingefuegt.
Durch den Primary Key auf `fixture_id` werden doppelte Eintraege verhindert.


In [15]:
with open("db/05_insert_matches.sql", "w") as f:
    for _, row in matches_df.iterrows():
        f.write(
            "INSERT INTO matches "
            "(fixture_id, date, season, venue, home_team, away_team, home_goals, away_goals) VALUES "
            f"({row.fixture_id}, '{row.date}', {row.season}, "
            f"{'NULL' if pd.isna(row.venue) else repr(row.venue)}, "
            f"{repr(row.home_team)}, {repr(row.away_team)}, "
            f"{row.home_goals}, {row.away_goals});\n"
        )


### Erstellung der Tabelle `match_team_stats`

Die Tabelle speichert jede Statistik als einzelne Zeile pro Team und Match.


In [16]:
create_stats_table = """
CREATE TABLE IF NOT EXISTS match_team_stats (
    fixture_id INTEGER NOT NULL,
    team_name TEXT NOT NULL,
    side TEXT CHECK (side IN ('home', 'away')),
    stat_name TEXT NOT NULL,
    stat_value NUMERIC,

    PRIMARY KEY (fixture_id, team_name, side, stat_name),

    CONSTRAINT fk_stats_match
        FOREIGN KEY (fixture_id) REFERENCES matches(fixture_id)
);

"""
with open("db/03_create_match_team_stats.sql", "w") as f:
    f.write(create_stats_table)


### Transformation der Statistiken ins Long-Format

Für jedes Match und jedes Team werden die Statistikspalten
in einzelne Zeilen überführt.


In [17]:
stats_rows = []

for _, row in df.iterrows():
    fixture_id = row["fixture_id"]

    # Home-Team Statistiken
    for stat in stat_columns:
        stats_rows.append(
            {
                "fixture_id": fixture_id,
                "team_name": row["home_team"],
                "side": "home",
                "stat_name": stat,
                "stat_value": row[f"home_{stat}"],
            }
        )

    # Away-Team Statistiken
    for stat in stat_columns:
        stats_rows.append(
            {
                "fixture_id": fixture_id,
                "team_name": row["away_team"],
                "side": "away",
                "stat_name": stat,
                "stat_value": row[f"away_{stat}"],
            }
        )

stats_long_df = pd.DataFrame(stats_rows)


### Einfügen der Match-Statistiken

Die transformierten Statistikdaten werden in die Tabelle
`match_team_stats` eingefügt.


In [18]:
len(stats_long_df)


36480

In [19]:
with open("db/06_insert_match_team_stats.sql", "w") as f:
    for _, row in stats_long_df.iterrows():
        f.write(
            f"INSERT INTO match_team_stats "
            f"(fixture_id, team_name, side, stat_name, stat_value) VALUES "
            f"({row.fixture_id}, '{row.team_name}', '{row.side}', "
            f"'{row.stat_name}', {row.stat_value});\n"
        )


## Hinweis zur Datenbankanbindung

Die SQL-Statements zur Schema-Erstellung und Datenpersistenz
werden bewusst als `.sql`-Dateien im Ordner `/db` erzeugt
und manuell in PostgreSQL (Neon) ausgeführt.

Ein direkter Insert aus Python wurde vermieden,
da dies in Cloud-Datenbanken ineffizient ist.
